In [8]:
import time
import numpy as np
from apsuite.asparams import BO_MOM_COMPACT
from mathphys.functions import save_pickle, load_pickle
from siriuspy.devices import SOFB, RFGen, EVG

Creating devices

In [14]:
sofb = SOFB(SOFB.DEVICES.BO)
rfgen = RFGen()
evg = EVG()

Useful functions

In [7]:
def get_orb():
    """."""
    sofb.cmd_reset()
    sofb.wait_buffer()
    return np.array([sofb.trajx, sofb.trajy]).transpose()

def deltaen(deltaf, freq0):
    """."""
    return -deltaf/freq0/BO_MOM_COMPACT

Saving RF initial state

In [15]:
f0 = rfgen.frequency

Creating data dict

In [ ]:
data = dict()

deltaf = 0
freqp = f0 + deltaf
freqm = f0 - deltaf
data['freqs'] = [freqm, freq0, freqp]

Does the measure

In [ ]:
# Getting default orbit
evg.cmd_turn_on_injection()
data['orb0'] = get_orb()

time.sleep(2)
# Increasing the frequency
rfgen.frequency = freqp
evg.cmd_turn_on_injection()
data['orbp'] = get_orb()

time.sleep(2)
# Decreasing the frequency
rfgen.frequency = freqm
evg.cmd_turn_on_injection()
data['orbm'] = get_orb()

time.sleep(2)
# Restore deafault frequency
rfgen.frequency = f0

Computing dispersion

In [ ]:
denergy = deltaen(deltaf, f0)
dispersion = (data['orbp'] - data['orbm'])/2/denergy
dispersion *= 1e-4  # Converting from um to cm
data['dispersion'] = dispersion
data['delta_energy'] = denergy

Saving data

In [ ]:
save_pickle(f"bo_disp_deltaf={deltaf:.2f}")